# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import PicklingError
from joblib import dump, load
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, recall_score, classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, matthews_corrcoef, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from datetime import datetime
from tensorflow import keras
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.utils.data import evaluate_print
from IPython.display import display, Markdown
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Settings
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "device_sessions_single_user"
TIME_BUCKETS  = [
    "7d", 
    "2d", 
    "1d", 
    "12h",
    "6h", 
    "1h"
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def add_count_and_threat(x):
    avg_session_duration = x.session_duration.mean()
    threat = 1 if 1 in x.threat.values else 0
    return pd.Series([len(x), avg_session_duration, threat], index=["count", "avg_session_duration", "threat"])


def train_classifier_on_single_users(df, classifier, time_bucket, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in df.groupby("user"):

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            if len(group) == 1:
                classifier.set_params(**{"n_clusters": 1})

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])


            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group[["count", "avg_session_duration"]])
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group[["count", "avg_session_duration"]])
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, time_bucket, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.time_bucket = time_bucket
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, time_bucket in enumerate(TIME_BUCKETS, start=1):

        display(Markdown(f"# {i}/{len(TIME_BUCKETS)} - Bucket: {time_bucket}"))

        # Group the dataframe by users and time buckets
        tmp = df.groupby([df["user"], pd.Grouper(freq=time_bucket, key="date")]).apply(add_count_and_threat)
        tmp = tmp.reindex(pd.MultiIndex.from_product(tmp.index.levels))
        tmp = tmp.fillna(0)

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=tmp,
            classifier=classifier,
            time_bucket=time_bucket,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "device_sessions"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 24 different hyperparameter combinations
 Training Time: 2.5448    Inference Time: 0.7526    pAUC: 0.6930 ± 0.0061   Recall: 0.1609 ± 0.0216   TN: 18755.0   FP: 146.0   FN: 153.0   TP: 26.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 30.6714   Inference Time: 1.6081    pAUC: 0.7074 ± 0.0105   Recall: 0.1318 ± 0.0135   TN: 18739.0   FP: 162.0   FN: 152.0   TP: 27.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 38.0349   Inference Time: 5.2337    pAUC: 0.7153 ± 0.0043   Recall: 0.1128 ± 0.0108   TN: 18733.0   FP: 168.0   FN: 157.0   TP: 22.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 73.7003   Inference Time: 9.6799    pAUC: 0.7157 ± 0.0042   Recall: 0.1117 ± 0.0093   TN: 18732.0   FP: 169.0   FN: 158.0   TP: 21.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 2.6217    Inference Time: 0.7716    pAUC: 0.6930 ± 0.0061   Recall: 0.1609 ± 0.0216   TN: 18755.0   FP: 146.0   FN: 153.0   TP: 26.0   

# 2/6 - Bucket: 2d

Testing 24 different hyperparameter combinations
 Training Time: 3.8951    Inference Time: 1.1739    pAUC: 0.6478 ± 0.0155   Recall: 0.1377 ± 0.0244   TN: 65654.0   FP: 336.0   FN: 228.0   TP: 32.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 33.1926   Inference Time: 2.6248    pAUC: 0.7178 ± 0.0067   Recall: 0.2400 ± 0.0093   TN: 65389.0   FP: 601.0   FN: 199.0   TP: 61.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 50.3124   Inference Time: 8.7360    pAUC: 0.7279 ± 0.0025   Recall: 0.2200 ± 0.0086   TN: 65389.0   FP: 601.0   FN: 199.0   TP: 61.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 90.4020   Inference Time: 16.3666   pAUC: 0.7268 ± 0.0022   Recall: 0.2192 ± 0.0064   TN: 65383.0   FP: 607.0   FN: 205.0   TP: 55.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 3.8936    Inference Time: 1.1685    pAUC: 0.7028 ± 0.0074   Recall: 0.2400 ± 0.0176   TN: 65488.0   FP: 502.0   FN: 195.0   TP: 65.0   

# 3/6 - Bucket: 1d

Testing 24 different hyperparameter combinations
 Training Time: 2.7330    Inference Time: 0.8228    pAUC: 0.6428 ± 0.0186   Recall: 0.2007 ± 0.0333   TN: 131801   FP: 414     FN: 220   TP: 65    Params: {'max_samples': 128, 'n_estimators': 1}


# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 28 different hyperparameter combinations
 Training Time: 2.8131    Inference Time: 0.7175    pAUC: 0.5246 ± 0.0000   Recall: 0.0503 ± 0.0000   TN: 18719.0   FP: 182.0   FN: 170.0   TP: 9.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 3.8436    Inference Time: 0.8095    pAUC: 0.5222 ± 0.0000   Recall: 0.0503 ± 0.0000   TN: 18737.0   FP: 164.0   FN: 170.0   TP: 9.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 4.8837    Inference Time: 0.8985    pAUC: 0.5283 ± 0.0000   Recall: 0.0503 ± 0.0000   TN: 18720.0   FP: 181.0   FN: 170.0   TP: 9.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 5.7256    Inference Time: 0.9732    pAUC: 0.5323 ± 0.0000   Recall: 0.0559 ± 0.0000   TN: 18725.0   FP: 176.0   FN: 169.0   TP: 10.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.8957    Inference Time: 0.7517    pAUC: 0.5358 ± 0.0000   Recall: 0.0335 ± 0.0000   TN: 18706.0   FP: 195.0   FN: 173.0   TP: 6.0   Params: {'n_bins': 8, 'n_ra

# 2/6 - Bucket: 2d

Testing 28 different hyperparameter combinations
 Training Time: 2.8717    Inference Time: 0.7561    pAUC: 0.5381 ± 0.0000   Recall: 0.0385 ± 0.0000   TN: 65301.0   FP: 689.0   FN: 250.0   TP: 10.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 3.9432    Inference Time: 0.8650    pAUC: 0.5567 ± 0.0000   Recall: 0.0423 ± 0.0000   TN: 65352.0   FP: 638.0   FN: 249.0   TP: 11.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 5.0472    Inference Time: 0.9876    pAUC: 0.5486 ± 0.0000   Recall: 0.0462 ± 0.0000   TN: 65375.0   FP: 615.0   FN: 248.0   TP: 12.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 6.0982    Inference Time: 1.1168    pAUC: 0.5469 ± 0.0000   Recall: 0.0462 ± 0.0000   TN: 65366.0   FP: 624.0   FN: 248.0   TP: 12.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.8245    Inference Time: 0.7499    pAUC: 0.5265 ± 0.0000   Recall: 0.0192 ± 0.0000   TN: 65403.0   FP: 587.0   FN: 255.0   TP: 5.0   Params: {'n_bins': 8, 'n

# 3/6 - Bucket: 1d

...

Testing 28 different hyperparameter combinations
 Training Time: 3.1874    Inference Time: 0.9070    pAUC: 0.5590 ± 0.0000   Recall: 0.0377 ± 0.0000   TN: 261889.0   FP: 2634.0   FN: 459.0   TP: 18.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 5.0295    Inference Time: 1.2843    pAUC: 0.5692 ± 0.0000   Recall: 0.0482 ± 0.0000   TN: 262049.0   FP: 2474.0   FN: 454.0   TP: 23.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 5.9237    Inference Time: 1.4139    pAUC: 0.5828 ± 0.0000   Recall: 0.0440 ± 0.0000   TN: 261796.0   FP: 2727.0   FN: 456.0   TP: 21.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 7.2608    Inference Time: 1.6585    pAUC: 0.5816 ± 0.0000   Recall: 0.0440 ± 0.0000   TN: 261806.0   FP: 2717.0   FN: 456.0   TP: 21.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 3.1624    Inference Time: 0.9104    pAUC: 0.5578 ± 0.0000   Recall: 0.0210 ± 0.0000   TN: 261743.0   FP: 2780.0   FN: 467.0   TP: 10.0   Params: {'n_b

# 5/6 - Bucket: 6h

Testing 28 different hyperparameter combinations
 Training Time: 3.5563    Inference Time: 1.0591    pAUC: 0.6682 ± 0.0000   Recall: 0.0904 ± 0.0000   TN: 523987.0   FP: 5217.0   FN: 483.0   TP: 48.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 5.6025    Inference Time: 1.6050    pAUC: 0.6771 ± 0.0000   Recall: 0.0791 ± 0.0000   TN: 524189.0   FP: 5015.0   FN: 489.0   TP: 42.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.8712    Inference Time: 1.8918    pAUC: 0.6880 ± 0.0000   Recall: 0.1073 ± 0.0000   TN: 524084.0   FP: 5120.0   FN: 474.0   TP: 57.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.5190    Inference Time: 2.2808    pAUC: 0.6732 ± 0.0000   Recall: 0.0697 ± 0.0000   TN: 523989.0   FP: 5215.0   FN: 494.0   TP: 37.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 3.4757    Inference Time: 1.0661    pAUC: 0.6519 ± 0.0000   Recall: 0.0264 ± 0.0000   TN: 524168.0   FP: 5036.0   FN: 517.0   TP: 14.0   Params: {'n_b

# 6/6 - Bucket: 1h

Testing 28 different hyperparameter combinations
 Training Time: 9.4820    Inference Time: 4.2963    pAUC: 0.8945 ± 0.0000   Recall: 0.2036 ± 0.0000   TN: 3145897.0   FP: 31123.0   FN: 896.0   TP: 229.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 16.4929   Inference Time: 7.5104    pAUC: 0.9111 ± 0.0000   Recall: 0.3298 ± 0.0000   TN: 3145598.0   FP: 31422.0   FN: 754.0   TP: 371.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 23.4092   Inference Time: 10.7827   pAUC: 0.9111 ± 0.0000   Recall: 0.2427 ± 0.0000   TN: 3145972.0   FP: 31048.0   FN: 852.0   TP: 273.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 31.0148   Inference Time: 14.2527   pAUC: 0.9134 ± 0.0000   Recall: 0.3778 ± 0.0000   TN: 3145620.0   FP: 31400.0   FN: 700.0   TP: 425.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 9.3750    Inference Time: 4.3690    pAUC: 0.9199 ± 0.0000   Recall: 0.3840 ± 0.0000   TN: 3145214.0   FP: 31806.0   FN: 693.0   TP: 432.0 

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 2.1652    Inference Time: 0.9374    pAUC: 0.7241 ± 0.0000   Recall: 0.2961 ± 0.0000   TN: 18767    FP: 134     FN: 126   TP: 53    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7241 ± 0.0000
Recall: 0.2961 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 1.9311    Inference Time: 0.8821    pAUC: 0.8452 ± 0.0000   Recall: 0.3808 ± 0.0000   TN: 65424    FP: 566     FN: 161   TP: 99    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8452 ± 0.0000
Recall: 0.3808 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 2.0025    Inference Time: 1.0443    pAUC: 0.8137 ± 0.0000   Recall: 0.3375 ± 0.0000   TN: 262035   FP: 2488    FN: 316   TP: 161   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8137 ± 0.0000
Recall: 0.3375 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 2.1943    Inference Time: 1.2261    pAUC: 0.8583 ± 0.0000   Recall: 0.4294 ± 0.0000   TN: 524134   FP: 5070    FN: 303   TP: 228   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8583 ± 0.0000
Recall: 0.4294 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 3.5093    Inference Time: 3.4634    pAUC: 0.9327 ± 0.0000   Recall: 0.4098 ± 0.0000   TN: 3145703   FP: 31317   FN: 664   TP: 461   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9327 ± 0.0000
Recall: 0.4098 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 1.9228    Inference Time: 0.8224    pAUC: 0.7013 ± 0.0000   Recall: 0.2626 ± 0.0000   TN: 18737    FP: 164     FN: 132   TP: 47    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7013 ± 0.0000
Recall: 0.2626 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 1.9502    Inference Time: 0.8785    pAUC: 0.8274 ± 0.0000   Recall: 0.3462 ± 0.0000   TN: 65411    FP: 579     FN: 170   TP: 90    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8274 ± 0.0000
Recall: 0.3462 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 2.0114    Inference Time: 1.0482    pAUC: 0.8109 ± 0.0000   Recall: 0.3375 ± 0.0000   TN: 262034   FP: 2489    FN: 316   TP: 161   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8109 ± 0.0000
Recall: 0.3375 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 2.1697    Inference Time: 1.2361    pAUC: 0.8583 ± 0.0000   Recall: 0.4313 ± 0.0000   TN: 524138   FP: 5066    FN: 302   TP: 229   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8583 ± 0.0000
Recall: 0.4313 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 3.5889    Inference Time: 3.5537    pAUC: 0.9327 ± 0.0000   Recall: 0.4098 ± 0.0000   TN: 3145701   FP: 31319   FN: 664   TP: 461   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.9327 ± 0.0000
Recall: 0.4098 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [1, 2, 4],
    "alpha": [0.5, 0.6, 0.7, 0.8, 0.9],
    "beta": [2, 4, 8, 16, 32, 64],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 168 different hyperparameter combinations
 Training Time: 1.9259    Inference Time: 0.6563    pAUC: 0.6034 ± 0.0000   Recall: 0.1508 ± 0.0000   TN: 18737.0   FP: 164.0   FN: 152.0   TP: 27.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 1.8381    Inference Time: 0.6339    pAUC: 0.6034 ± 0.0000   Recall: 0.1508 ± 0.0000   TN: 18737.0   FP: 164.0   FN: 152.0   TP: 27.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 1.8188    Inference Time: 0.6278    pAUC: 0.6268 ± 0.0000   Recall: 0.2179 ± 0.0000   TN: 18750.0   FP: 151.0   FN: 140.0   TP: 39.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 1.8156    Inference Time: 0.6276    pAUC: 0.6268 ± 0.0000   Recall: 0.2179 ± 0.0000   TN: 18750.0   FP: 151.0   FN: 140.0   TP: 39.0   Params: {'n_compo

# 2/6 - Bucket: 2d

Testing 168 different hyperparameter combinations
 Training Time: 1.8136    Inference Time: 0.6288    pAUC: 0.6634 ± 0.0000   Recall: 0.2269 ± 0.0000   TN: 65386.0   FP: 604.0   FN: 201.0   TP: 59.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 1.8911    Inference Time: 0.6562    pAUC: 0.6634 ± 0.0000   Recall: 0.2269 ± 0.0000   TN: 65386.0   FP: 604.0   FN: 201.0   TP: 59.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 1.8207    Inference Time: 0.6313    pAUC: 0.7037 ± 0.0000   Recall: 0.2731 ± 0.0000   TN: 65399.0   FP: 591.0   FN: 189.0   TP: 71.0   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 1.8118    Inference Time: 0.6267    pAUC: 0.7037 ± 0.0000   Recall: 0.2731 ± 0.0000   TN: 65399.0   FP: 591.0   FN: 189.0   TP: 71.0   Params: {'n_compo

# 3/6 - Bucket: 1d

...

Testing 168 different hyperparameter combinations
 Training Time: 2.6127    Inference Time: 0.9314    pAUC: 0.7872 ± 0.0000   Recall: 0.4256 ± 0.0000   TN: 262075   FP: 2448    FN: 274   TP: 203   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 2.6653    Inference Time: 0.9428    pAUC: 0.7872 ± 0.0000   Recall: 0.4256 ± 0.0000   TN: 262075   FP: 2448    FN: 274   TP: 203   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 2.5930    Inference Time: 0.9098    pAUC: 0.7931 ± 0.0000   Recall: 0.3836 ± 0.0000   TN: 262055   FP: 2468    FN: 294   TP: 183   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.6803    Inference Time: 0.9438    pAUC: 0.7931 ± 0.0000   Recall: 0.3836 ± 0.0000   TN: 262055   FP: 2468    FN: 294   TP: 183   Params: {'n_components': 1, 'stan

# 5/6 - Bucket: 6h

Testing 168 different hyperparameter combinations
 Training Time: 2.7176    Inference Time: 0.9755    pAUC: 0.8675 ± 0.0000   Recall: 0.5593 ± 0.0000   TN: 524204   FP: 5000    FN: 234   TP: 297   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 2.6716    Inference Time: 0.9499    pAUC: 0.8675 ± 0.0000   Recall: 0.5593 ± 0.0000   TN: 524204   FP: 5000    FN: 234   TP: 297   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 2.6566    Inference Time: 0.9338    pAUC: 0.8685 ± 0.0000   Recall: 0.6008 ± 0.0000   TN: 524226   FP: 4978    FN: 212   TP: 319   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 2.6680    Inference Time: 0.9377    pAUC: 0.8685 ± 0.0000   Recall: 0.6008 ± 0.0000   TN: 524226   FP: 4978    FN: 212   TP: 319   Params: {'n_components': 1, 'stan

# 6/6 - Bucket: 1h

Testing 168 different hyperparameter combinations
 Training Time: 3.1490    Inference Time: 1.1864    pAUC: 0.9438 ± 0.0000   Recall: 0.4107 ± 0.0000   TN: 3145695   FP: 31325   FN: 663   TP: 462   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 3.1606    Inference Time: 1.1881    pAUC: 0.9438 ± 0.0000   Recall: 0.4107 ± 0.0000   TN: 3145695   FP: 31325   FN: 663   TP: 462   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 3.1413    Inference Time: 1.1791    pAUC: 0.9477 ± 0.0000   Recall: 0.4489 ± 0.0000   TN: 3145743   FP: 31277   FN: 620   TP: 505   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 3.1677    Inference Time: 1.1960    pAUC: 0.9477 ± 0.0000   Recall: 0.4489 ± 0.0000   TN: 3145743   FP: 31277   FN: 620   TP: 505   Params: {'n_components': 1, '

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 4/3 - Bucket: 12h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 1175.4734   Inference Time: 24.1542   pAUC: 0.6739 ± 0.0000   Recall: 0.0294 ± 0.0000   TN: 261887.0   FP: 2636.0   FN: 463.0   TP: 14.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 1157.2270   Inference Time: 25.3449   pAUC: 0.6739 ± 0.0000   Recall: 0.0294 ± 0.0000   TN: 261886.0   FP: 2637.0   FN: 463.0   TP: 14.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 1165.9417   Inference Time: 24.9755   pAUC: 0.6772 ± 0.0000   Recall: 0.0294 ± 0.0000   TN: 261886.0   FP: 2637.0   FN: 463.0   TP: 14.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 

# 5/3 - Bucket: 6h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 977.6770   Inference Time: 38.5186   pAUC: 0.6806 ± 0.0000   Recall: 0.0245 ± 0.0000   TN: 523921   FP: 5283    FN: 518   TP: 13    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 995.8466   Inference Time: 37.4876   pAUC: 0.6807 ± 0.0000   Recall: 0.0245 ± 0.0000   TN: 523956   FP: 5248    FN: 518   TP: 13    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 978.3453   Inference Time: 37.9531   pAUC: 0.6799 ± 0.0000   Recall: 0.0245 ± 0.0000   TN: 523919   FP: 5285    FN: 518   TP: 13    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 1006.8

# 6/3 - Bucket: 1h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 1075.8859   Inference Time: 145.5533   pAUC: 0.8354 ± 0.0000   Recall: 0.1556 ± 0.0000   TN: 3145676   FP: 31344   FN: 950   TP: 175   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 1112.3574   Inference Time: 149.4402   pAUC: 0.8399 ± 0.0000   Recall: 0.1547 ± 0.0000   TN: 3145415   FP: 31605   FN: 951   TP: 174   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 1120.2651   Inference Time: 149.8180   pAUC: 0.8375 ± 0.0000   Recall: 0.1556 ± 0.0000   TN: 3145414   FP: 31606   FN: 950   TP: 175   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Tim

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/5 - Bucket: 1d

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}: Invalid parameter n_clusters for estimator DeepSVDD(batch_size=16384,
     c=array([0.14706, 0.1    , 0.14686, 0.2966 , 0.10301, 0.1    , 0.2678 ,
       0.11154, 0.1    , 0.1    , 0.22436, 0.1    , 0.2133 , 0.161  ,
       0.1    , 0.1    , 0.1    , 0.1    , 0.1    , 0.25014, 0.36207,
       0.15932, 0.1    , 0.1    , 0.14887, 0.1    , 0.     , 0.1    ,
       0.40866, 0.     , 0.1    , 0.1    ], dtype=float32),
     callbacks=[<keras.callbacks.ReduceLROnPlateau object at 0x7ff29fcbaa10>, <keras.callbacks.EarlyStopping object at 0x7ff29e012cd0>],
     contamination=0.07, dropout_rate=0.2, epochs=100,
     hidden_activation='relu', hidden_neurons=[64, 32], l2_regularizer=0.1,
     optimizer=<keras.optimizer_v2

AttributeError: ignored